# Age Prediction with Deep Learning

## Importing Essential Libraries

In [1]:
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import cifar10
import cv2
import pandas as pd
import os 
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, Input, MaxPooling2D,Dropout,BatchNormalization,Reshape
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, Input, MaxPooling2D,Dropout,BatchNormalization,Reshape
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import seaborn as sns
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.metrics import accuracy_score
import math
from tensorflow.keras.models import load_model
from sklearn.metrics import r2_score
from tensorflow.keras import regularizers

## Import Data

In [2]:
df = pd.read_csv("train.csv")

## Data Analysis

In [3]:
df.shape

(17999, 3)

In [4]:
df.head()

,id,filename,age
0,1,1.jpg,40
1,2,2.jpg,54
2,3,3.jpg,51
3,4,4.jpg,35
4,5,5.jpg,46


In [5]:
df.isnull().sum()

id          0
filename    0
age         0
dtype: int64

In [6]:
df['filename']="applications-of-deep-learning-wustl-spring-2024/faces-age/"+df['filename']

In [7]:
df.head()

,id,filename,age
0,1,applications-of-deep-learning-wustl-spring-202...,40
1,2,applications-of-deep-learning-wustl-spring-202...,54
2,3,applications-of-deep-learning-wustl-spring-202...,51
3,4,applications-of-deep-learning-wustl-spring-202...,35
4,5,applications-of-deep-learning-wustl-spring-202...,46


In [8]:
df["filename"]

0        applications-of-deep-learning-wustl-spring-202...
1        applications-of-deep-learning-wustl-spring-202...
2        applications-of-deep-learning-wustl-spring-202...
3        applications-of-deep-learning-wustl-spring-202...
4        applications-of-deep-learning-wustl-spring-202...
                               ...                        
17994    applications-of-deep-learning-wustl-spring-202...
17995    applications-of-deep-learning-wustl-spring-202...
17996    applications-of-deep-learning-wustl-spring-202...
17997    applications-of-deep-learning-wustl-spring-202...
17998    applications-of-deep-learning-wustl-spring-202...
Name: filename, Length: 17999, dtype: object

## Image Process

In [4]:
import os
import cv2
import numpy as np
import pandas as pd

# Load dataframe and image filenames
df = pd.read_csv("train.csv")  # Ensure correct CSV file path

image_data = []
for img_name in df['filename']:
    # Construct platform-independent file path
    full_path = os.path.join("faces-age", img_name)
    
    # Read image file
    img = cv2.imread(full_path)
    
    # Handle missing/unreadable files
    if img is None:
        print(f"ERROR: {full_path} not found or unreadable!")
        continue
    
    try:
        # Convert color space from BGR to RGB
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        # Resize and normalize image
        img = cv2.resize(img, (128, 128))
        img = img / 255.0
        
        image_data.append(img)
        
    except Exception as e:
        print(f"ERROR: Processing failed for {full_path}: {str(e)}")

# Convert list to numpy array
image_data = np.array(image_data)
print(f"Successfully loaded images: {len(image_data)}")

Successfully loaded images: 17999


In [10]:
y = df["age"]

In [11]:
y.shape

(17999,)

## Modelling

In [12]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=.2,random_state=42)

In [13]:
model = Sequential()

model.add(Input(shape=(128, 128, 3)))

model.add(Conv2D(32, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64, kernel_size=(3,3), activation='relu',kernel_regularizer=regularizers.l1_l2(l1=0.01,l2=0.01)))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(128, kernel_size=(3,3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(256, kernel_size=(3,3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())

model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))  
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))

model.add(Dense(1) ) 

model.compile(optimizer='adam', loss='mse', metrics=['mae', 'mse'])

In [ ]:
history = model.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=100,batch_size=8,verbose=1)

Epoch 1/100
  35/1800 ━━━━━━━━━━━━━━━━━━━━ 3:15 111ms/step - loss: 734.3027 - mae: 22.1034 - mse: 726.3375  

In [ ]:
model.save("model.h5")